# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
# Llenar credenciales para poder reroducir gráficas en Plotly:
# plotly.tools.set_credentials_file(username='', api_key='')
pd.options.display.max_colwidth = 300
dir_datos = 'D:/datos/presupuesto_mexico'
%matplotlib inline

# Pre-procesando bases

## Precios

In [2]:
deflactor = pd.read_excel('datos/inflacion_acumulada_1970_2018.xls', header=3, skipfooter=5)\
    .assign(year=lambda x: x['Periodo'].str[0:4].astype(int),
           mes=lambda x: x['Periodo'].str[5:].astype(int))\
    .rename(columns={'Índice general ': 'inpc'})\
    .query('mes==12')\
    .sort_values('year')\
    .set_index('year')\
    .assign(indice=lambda x: x['inpc'].apply(lambda x: (1+x/100)).cumprod())\
    .assign(deflactor=lambda x: x.loc[2018, 'indice']/x['indice'])\
    .reset_index()\
    [['year', 'deflactor']]\
    .append(pd.DataFrame([{'year': 2019, 'deflactor': 0.9671}]), sort=True)
deflactor.to_csv('datos/deflactor_2018.csv', index=False)

## Presupuesto

In [ ]:
df = pd.concat([pd.read_csv(f'{dir_datos}/{y}.csv') for y in range(2013, 2020)])
df.to_csv(f'{dir_datos}/presupuesto_2013_2019.csv', index=False)

In [3]:
deflactor = pd.read_csv('datos/deflactor_2018.csv').query('year>=2013')

df = pd.read_csv(f'{dir_datos}/presupuesto_2013_2019.csv')\
    .assign(ID_PP=lambda x: x['ID_PP'].astype(str).str.zfill(3))\
    .assign(id_pp=lambda x: x['ID_MODALIDAD']+x['ID_PP'])\
    .set_index('CICLO')
df['aprobado'] = (df['MONTO_APROBADO']*deflactor.set_index('year')['deflactor'])
df.reset_index(inplace=True)


dh = df.loc[df['DESC_PP'].str.contains('derechos humanos', case=False)|
      df['DESC_RAMO'].str.lower().str.contains('derechos humanos', case=False)]\
    .assign(ID_PP=lambda x: x['ID_PP'].astype(str).str.zfill(3))\
    .assign(id_pp=lambda x: x['ID_MODALIDAD']+x['ID_PP'])\
    .set_index('CICLO')

dh['aprobado'] = (dh['MONTO_APROBADO']*deflactor.set_index('year')['deflactor'])
dh['ejercicio'] = dh['MONTO_EJERCICIO']*deflactor.set_index('year')['deflactor']
dh['ejercido'] = dh['MONTO_EJERCIDO']*deflactor.set_index('year')['deflactor']
dh['modificado'] = dh['MONTO_MODIFICADO']*deflactor.set_index('year')['deflactor']
dh['devengado'] = dh['MONTO_DEVENGADO']*deflactor.set_index('year')['deflactor']
dh['pagado'] = dh['MONTO_PAGADO']*deflactor.set_index('year')['deflactor']
dh.to_csv(f'presupuesto_ddhh_2013_2019.csv', index=True)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (19,28,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Entradas y salidas de PP

In [ ]:
# Entradas y salidas de PPs
writer = pd.ExcelWriter('entradas_salidas_pp.xlsx', engine='xlsxwriter')
dh.groupby(['CICLO', 'DESC_RAMO']).apply(lambda x: set(x['id_pp'].unique().tolist())).to_frame('pp')\
    .assign(entran=lambda z: z.groupby('DESC_RAMO').transform(lambda y: y.diff(1)),
            salen=lambda z: z.groupby('DESC_RAMO').transform(lambda y: y.diff(-1).shift(1)))\
    .to_excel(writer, sheet_name='year_ramo')
    #.assign(acumulado=lambda z: z.groupby(['DESC_RAMO'])['pp'].apply(lambda v: v.apply(list).cumsum().apply(set).shift(2)))\
    #.assign(reingreso=lambda u: u['entran']-u['entran'])
dh.groupby(['CICLO']).apply(lambda x: set(x['id_pp'].unique().tolist())).to_frame('pp')\
    .assign(entran=lambda z: z['pp'].diff(1),
            salen=lambda z: z['pp'].diff(-1).shift(1))\
    .to_excel(writer, sheet_name='year')    
writer.save()

# Análisis

## Leyendo datos

In [6]:
dh = pd.read_csv('datos/presupuesto_ddhh_2013_2019.csv') / 1000000
df = pd.read_csv(dir_datos+'/presupuesto_2013_2019.csv')
deflactor = pd.read_csv('datos/deflactor_2018.csv').query('year>=2013')
dh.set_index('CICLO', inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning:

Columns (19,28,31) have mixed types. Specify dtype option on import or set low_memory=False.



## Total

In [10]:
# Add data
dh_total = dh.groupby(['CICLO'])['aprobado'].sum().reset_index()
# Create and style traces
data = [go.Scatter(
    x = dh_total['CICLO'],
    y = dh_total['aprobado']/1000000,
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4)
)]


# Edit the layout
layout = dict(title = 'Gasto en programas presupuestarios de Derechos Humanos',
              xaxis = dict(title = ''),
              yaxis = dict(title = 'Millone de pesos (2018)',
                          rangemode='tozero', autorange=True),
              )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='presup-ddhh-total')

## Por ramo

In [11]:
dicc_ramos = {'Comisión Nacional de los Derechos Humanos': 'CNDH',
             'Defensa Nacional': 'Defensa',
              'Instituto Nacional de Transparencia, Acceso a la Información y Protección de Datos Personales': 'INAI',
              'Gobernación': 'SEGOB',
             'Procuraduría General de la República': 'PGR',
             'Relaciones Exteriores': 'SRE',
             'Seguridad Pública': 'SSP'}
dh_ramo = dh.query('CICLO==2019').groupby(['DESC_RAMO'])[['aprobado']].sum()
data_ramo = (100*(dh_ramo/dh_ramo.sum())).reset_index().replace({'DESC_RAMO': dicc_ramos})

fig = {
  "data": [
    {
      "values": data_ramo['aprobado'].tolist(),
      "labels": data_ramo['DESC_RAMO'].tolist(),
      "domain": {"x": [0, 1]},
      "hoverinfo":"label+percent",
      "hole": .6,
      "type": "pie"
    }],
     "layout": {
        "title":"Distribución del presupuesto en DDHH por ramo, 2019",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Participación <br> %",
                "x": 0.5,
                "y": 0.5
            },
        ]
    }
}
py.iplot(fig, filename='presup-ddhh-ramos')

## Gasto en programas

In [12]:
[x for x in dh.query('CICLO==[2018, 2019]').DESC_PP.unique() if re.search('mujer|niñ|lgbt|migra|period|sex|desap|indig', x, re.I)]

['Atender asuntos relacionados con las personas migrantes',
 'Atender asuntos relacionados con personas reportadas como desaparecidas, extraviadas, ausentes y fallecidas no identificadas',
 'Atender asuntos relacionados con niñas, niños y adolescentes',
 'Atender asuntos relacionados con sexualidad, salud y VIH',
 'Promover, divulgar, dar seguimiento, evaluar y monitorear la política nacional en materia de igualdad entre mujeres y hombres, y atender asuntos de la mujer',
 'Promover el respeto de los derechos humanos de periodistas y personas defensores de derechos humanos',
 'Atender asuntos relacionados con personas reportadas como desaparecidas y no localizadas',
 'Atender asuntos relacionados con niñas, niños y adolescentes.',
 'Atender asuntos relacionados con Sexualidad, Salud y VIH',
 'Realizar la promoción y observancia en el monitoreo, seguimiento y evaluación del impacto de la política nacional en materia de igualdad entre mujeres y hombres.',
 'Promover el respeto de los Dere

In [115]:
dicc_replace_hm = {'realizar la promoción y observancia en el monitoreo, seguimiento y evaluación del impacto de la política nacional en materia de igualdad entre mujeres y hombres':
                  'promover, divulgar, dar seguimiento, evaluar y monitorear la política nacional en materia de igualdad entre mujeres y hombres, y atender asuntos de la mujer',
                  'atender asuntos relacionados con personas reportadas como desaparecidas, extraviadas, ausentes y fallecidas no identificadas':
                   'atender asuntos relacionados con personas reportadas como desaparecidas y no localizadas'}
pps_dh = [x for x in dh.query('CICLO==[2018, 2019]').DESC_PP.unique() if re.search('mujer|niñ|lgbt|migra|period|sex|desap', x, re.I)]
dh_pp_cndh = dh.query('DESC_PP==@pps_dh & CICLO==[2018, 2019]')\
    .assign(pp=lambda x: (x['DESC_PP'].str.replace('.', '').str.lower())\
           .replace(dicc_replace_hm))\
    .groupby(['CICLO', 'pp']).sum()[['aprobado']]\
    .unstack(0).reset_index().sort_values(('aprobado', 2018))
dh_pp_cndh['diff'] = dh_pp_cndh['aprobado'][2018]-dh_pp_cndh['aprobado'][2019]
dicc_pp_cndh = {'atender asuntos relacionados con sexualidad, salud y vih': 'Sexualidad,<br> salud<br> y VIH',
               'atender asuntos relacionados con niñas, niños y adolescentes': 'Niñas,<br> niños y <br> adolescentes',
               'promover el respeto de los derechos humanos de periodistas y personas defensores de derechos humanos': 'Periodistas <br> y defensores<br> de DDHH',
               'promover, divulgar, dar seguimiento, evaluar y monitorear la política nacional en materia de igualdad entre mujeres y hombres, y atender asuntos de la mujer': 'Igualdad<br> de género',
               'atender asuntos relacionados con las personas migrantes': 'Personas <br>migrantes',
               'atender asuntos relacionados con personas reportadas como desaparecidas y no localizadas': 'Personas<br> desaparecidas'}

In [121]:
dh_pp_cndh['var_procentual'] = 100*(dh_pp_cndh['diff']/dh_pp_cndh['aprobado'][2018])
dh_pp_cndh


3    0.217453
1    0.456025
4    0.172772
2    0.244682
5    0.100800
0    0.082172
dtype: float64

In [322]:
x = dh_pp_cndh['pp'].map(dicc_pp_cndh)
trace1 = {
  'x': dh_pp_cndh['aprobado'][2019],
  'y': x,
  'hovertext': '2019',
  "hoverinfo":"text",
  'name': '2019',
  'type': 'bar',
  'orientation': 'h',
  'marker': dict(
        color='rgba(55, 128, 191, 0.7)',
        line=dict(
            color='rgba(55, 128, 191, 1.0)',
            width=2,
        )
    )
};
trace2 = {
  'x': dh_pp_cndh['aprobado'][2018],
  'y': x,
  'hovertext': '2018',
  "hoverinfo":"text",
  'name': '2018',
  'type': 'bar',
  'orientation': 'h',
    'marker': dict(
        color='rgba(219, 64, 82, 0.7)',
        line=dict(
            color='rgba(219, 64, 82, 1.0)',
            width=2,
        )
    )
};

data = [trace1, trace2];
layout = {
  'xaxis': {'title': ''},
  'xaxis': {'title': 'Millones de pesos (a precios de 2018 )'},
  'title': 'Gasto en programas de población específica de la CNDH'
};
py.iplot({'data': data, 'layout': layout}, filename='presup-cndh')

## Gasto en ddhh en otros ramos

In [326]:
dh_pp_otros= dh.query('ID_RAMO==[7, 4, 17] & CICLO==[2018, 2019]')\
    .assign(pp=lambda x: (x['DESC_PP'].str.lower()))\
    .groupby(['CICLO', 'pp']).sum()[['aprobado']]\
    .unstack(0)\
    .reset_index()\
    .sort_values(('aprobado', 2019), ascending=True)
dh_pp_otros['diff'] = dh_pp_otros['aprobado'][2018]-dh_pp_otros['aprobado'][2019]
dicc_pp_otros = {'derechos humanos': 'Derechos<br> Humanos<br> (Defensa)',
               'programa de derechos humanos': 'Programa <br> DDHH <br> (SEGOB)',
               'promover la protección de los derechos humanos y prevenir la discriminación': 'Prevenir <br> discriminación <br> (SEGOB)',
               'promoción del respeto a los derechos humanos y atención a víctimas del delito': 'Respeto<br> victimas <br> delito <br> (PGR)'}
x = dh_pp_otros['pp'].map(dicc_pp_otros)
trace1 = {
  'x': dh_pp_otros['aprobado'][2019],
  'y': x,
  'hovertext': '2019',
  "hoverinfo":"text",
  'name': '2019',
  'type': 'bar',
  'orientation': 'h',
  'marker': dict(
        color='rgba(55, 128, 191, 0.7)',
        line=dict(
            color='rgba(55, 128, 191, 1.0)',
            width=2,
        )
    )
};
trace2 = {
  'x': dh_pp_otros['aprobado'][2018],
  'y': x,
  'hovertext': '2018',
  "hoverinfo":"text",
  'name': '2018',
  'type': 'bar',
  'orientation': 'h',
    'marker': dict(
        color='rgba(219, 64, 82, 0.7)',
        line=dict(
            color='rgba(219, 64, 82, 1.0)',
            width=2,
        )
    )
};

data = [trace1, trace2];
layout = {
  'xaxis': {'title': ''},
  'xaxis': {'title': 'Millones de pesos (a precios de 2018 )'},
  'title': 'Gasto en programas de DDHH, en los ramos Defensa, SEGOB y PGR',
};
py.iplot({'data': data, 'layout': layout}, filename='presup-ddhh-otros')

## Gasto por entidad

In [14]:
dh.query('CICLO==2019')\
    .groupby('ENTIDAD_FEDERATIVA')['aprobado'].sum()/1000000

ENTIDAD_FEDERATIVA
Aguascalientes            7.732055
Baja California          10.136158
Baja California Sur       6.378394
Chiapas                  19.735329
Chihuahua                11.167462
Ciudad de México       2406.484846
Coahuila                  9.090201
Guerrero                  7.626391
Michoacán                 6.467911
Oaxaca                    8.891916
San Luis Potosí           9.010551
Sonora                   12.075835
Tabasco                   9.457116
Tamaulipas               11.411739
Veracruz                  8.895115
Yucatán                   8.027444
Name: aprobado, dtype: float64

In [15]:
dh_ent = dh.query('CICLO==2019 & ENTIDAD_FEDERATIVA!="Ciudad de México"')\
    .replace({'ENTIDAD_FEDERATIVA': {'Baja California Sur': 'BCS'}})\
    .groupby('ENTIDAD_FEDERATIVA')['aprobado'].sum()\
    .reset_index()\
    .sort_values('aprobado', ascending=False)
    
trace1 = {
  'x': dh_ent['ENTIDAD_FEDERATIVA'],
  'y': dh_ent['aprobado']/1000000,
  'type': 'bar',
  'orientation': 'v',
  'marker': dict(
        color='rgba(55, 128, 191, 0.7)',
        line=dict(
            color='rgba(55, 128, 191, 1.0)',
            width=2,
        )
    )
}

data = [trace1];
layout = {
  'xaxis': {'title': ''},
  'yaxis': {'title': 'Millones de pesos (a precios de 2018 )'},
  'title': 'Gasto en programas presupuestales de DDHH, por entidad, 2019',
  'annotations': [
        dict(
            x=0,
            y=-0.25,
            showarrow=False,
            text='Nota: excluimos la Ciudad de México para evitar problemas con la escala.',
            xref='paper',
            yref='paper'
        )]
};
py.iplot({'data': data, 'layout': layout}, filename='presup-ddhh-entidad')

In [309]:
dh.keys()

Index(['DESC_AI', 'DESC_CAPITULO', 'DESC_CONCEPTO', 'DESC_FF', 'DESC_FUNCION',
       'DESC_GPO_FUNCIONAL', 'DESC_MODALIDAD', 'DESC_PARTIDA_ESPECIFICA',
       'DESC_PARTIDA_GENERICA', 'DESC_PP', 'DESC_RAMO', 'DESC_SUBFUNCION',
       'DESC_TIPOGASTO', 'DESC_UR', 'ENTIDAD_FEDERATIVA', 'GPO_FUNCIONAL',
       'ID_AI', 'ID_CAPITULO', 'ID_CLAVE_CARTERA', 'ID_CONCEPTO',
       'ID_ENTIDAD_FEDERATIVA', 'ID_FF', 'ID_FUNCION', 'ID_MODALIDAD',
       'ID_PARTIDA_ESPECIFICA', 'ID_PARTIDA_GENERICA', 'ID_PP', 'ID_RAMO',
       'ID_SUBFUNCION', 'ID_TIPOGASTO', 'ID_UR', 'MONTO_ADEFAS',
       'MONTO_APROBADO', 'MONTO_DEVENGADO', 'MONTO_EJERCICIO',
       'MONTO_EJERCIDO', 'MONTO_MODIFICADO', 'MONTO_PAGADO', 'id_pp',
       'aprobado', 'ejercicio', 'ejercido', 'modificado', 'devengado',
       'pagado'],
      dtype='object')

## Por concepto

In [46]:
dh_conc = dh.assign(concepto=lambda x: (x['DESC_CONCEPTO'].str.replace('.', '').str.lower()))\
    .query('CICLO==2019')\
    .groupby(['concepto']).sum()\
    .sort_values('aprobado', ascending=False)\
    [['aprobado']]\
    .reset_index()\
    .iloc[0:10]
    
trace1 = {
  'x': dh_conc['concepto'],
  'y': dh_ent['aprobado']/1000000,
  'type': 'bar',
  'orientation': 'v',
  'marker': dict(
        color='rgba(55, 128, 191, 0.7)',
        line=dict(
            color='rgba(55, 128, 191, 1.0)',
            width=2,
        )
    )
}

data = [trace1];
layout = {
  'xaxis': {'title': ''},
  'yaxis': {'title': 'Millones de pesos (a precios de 2018 )'},
  'title': 'Top 10 conceptos de gasto en programas presupuestales de DDHH, 2019',
};
py.iplot({'data': data, 'layout': layout}, filename='presup-ddhh-concepto')

In [32]:
df_conc = df.assign(concepto=lambda x: (x['DESC_CONCEPTO'].str.replace('.', '').str.lower()))\
    .groupby(['concepto']).sum()

In [82]:
dh_conc

,concepto,aprobado
0,otras prestaciones sociales y económicas,8.760424e+08
1,remuneraciones al personal de carácter permanente,2.851699e+08
2,remuneraciones adicionales y especiales,2.495595e+08
3,"transferencias a fideicomisos, mandatos y otros análogos",2.007700e+08
4,"servicios profesionales, científicos, técnicos y otros servicios",1.896900e+08
5,seguridad social,1.722944e+08
6,pago de estímulos a servidores públicos,1.509073e+08
7,otros servicios generales,1.071462e+08
8,servicios de arrendamiento,6.900276e+07
9,servicios de traslado y viáticos,5.319502e+07


## Porcentaje que representa el gasto en DDHH como porcentaje del presupuesto total

In [71]:
pres_total = (((df.groupby(['CICLO']).sum()['MONTO_APROBADO'])*deflactor.set_index('year')['deflactor'])/1000000000000).reset_index()
prop_dh = (100*(dh.groupby(['CICLO']).sum()['aprobado']/((df.groupby(['CICLO']).sum()['MONTO_APROBADO'])*deflactor.set_index('year')['deflactor'])))\
.reset_index()

In [78]:
trace1 = go.Scatter(
    x=prop_dh['CICLO'],
    y=prop_dh[0],
    name='Presupuesto en DDHH <br> (eje izquierdo)'
)
trace2 = go.Scatter(
    x=pres_total['CICLO'],
    y=pres_total[0],
    name='Presupuesto total <br> (eje derecho)',
    yaxis='y2'
)
data = [trace1, trace2]
layout = go.Layout(
    title='Proporción que representan los programas presupuestales en DDHH <br> en el presupuesto total',
    yaxis=dict(
        title='Porcentaje del presupuesto total'
    ),
    yaxis2=dict(
        title='Billones de pesos <br>(a precios de 2018)',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='presup-ddhh-prop')

## Recomendaciones CNDH

In [81]:
#http://www.cndh.org.mx/Cifras_sobre_actividades_CNDH
recom = pd.read_excel('datos/recomendaciones_CNDH.xlsx')
   
trace1 = {
  'x': recom['year'],
  'y': recom['recomendaciones'],
  'type': 'bar',
  'orientation': 'v',
  'marker': dict(
        color='rgba(55, 128, 191, 0.7)',
        line=dict(
            color='rgba(55, 128, 191, 1.0)',
            width=2,
        )
    )
}

data = [trace1];
layout = {
  'xaxis': {'title': ''},
  'yaxis': {'title': 'Número de recomendaciones'},
  'title': 'Recomendaciones emitidas por la CNDH',
};
py.iplot({'data': data, 'layout': layout}, filename='recomendaciones-cndh')

# Presupuesto por ramos

In [92]:
presup_ramos_year = (dh.groupby(['DESC_RAMO', 'CICLO'])[['aprobado']].sum()/1000000).reset_index()\
    .assign(ramo=lambda x: x['DESC_RAMO'].map(dicc_ramos))
presup_ramos_year.to_csv('datos/presup_ramo_year.csv', index=False)

In [94]:
dh.keys()

Index(['DESC_AI', 'DESC_CAPITULO', 'DESC_CONCEPTO', 'DESC_FF', 'DESC_FUNCION',
       'DESC_GPO_FUNCIONAL', 'DESC_MODALIDAD', 'DESC_PARTIDA_ESPECIFICA',
       'DESC_PARTIDA_GENERICA', 'DESC_PP', 'DESC_RAMO', 'DESC_SUBFUNCION',
       'DESC_TIPOGASTO', 'DESC_UR', 'ENTIDAD_FEDERATIVA', 'GPO_FUNCIONAL',
       'ID_AI', 'ID_CAPITULO', 'ID_CLAVE_CARTERA', 'ID_CONCEPTO',
       'ID_ENTIDAD_FEDERATIVA', 'ID_FF', 'ID_FUNCION', 'ID_MODALIDAD',
       'ID_PARTIDA_ESPECIFICA', 'ID_PARTIDA_GENERICA', 'ID_PP', 'ID_RAMO',
       'ID_SUBFUNCION', 'ID_TIPOGASTO', 'ID_UR', 'MONTO_ADEFAS',
       'MONTO_APROBADO', 'MONTO_DEVENGADO', 'MONTO_EJERCICIO',
       'MONTO_EJERCIDO', 'MONTO_MODIFICADO', 'MONTO_PAGADO', 'id_pp',
       'aprobado', 'ejercicio', 'ejercido', 'modificado', 'devengado',
       'pagado'],
      dtype='object')

In [113]:
presup_tipogasto_year = (dh.groupby(['DESC_RAMO', 'CICLO', 'DESC_TIPOGASTO'])[['aprobado']].sum()/1000000).reset_index()\
    .assign(ramo=lambda x: x['DESC_RAMO'].map(dicc_ramos))
presup_tipogasto_year.to_csv('datos/presup_tipogasto_year.csv', index=False)

In [122]:
presup_tipogasto_year

,DESC_RAMO,CICLO,DESC_TIPOGASTO,aprobado,ramo
0,Comisión Nacional de los Derechos Humanos,2013,Gasto corriente,1620.280580,CNDH
1,Comisión Nacional de los Derechos Humanos,2013,Gasto de capital diferente de obra pública,52.829964,CNDH
2,Comisión Nacional de los Derechos Humanos,2013,Gasto de obra pública,17.917552,CNDH
3,Comisión Nacional de los Derechos Humanos,2014,Gasto corriente,1609.593161,CNDH
4,Comisión Nacional de los Derechos Humanos,2014,Gasto de capital diferente de obra pública,45.511605,CNDH
5,Comisión Nacional de los Derechos Humanos,2014,Gasto de obra pública,18.376163,CNDH
6,Comisión Nacional de los Derechos Humanos,2015,Gasto corriente,1644.186498,CNDH
7,Comisión Nacional de los Derechos Humanos,2015,Gasto de capital diferente de obra pública,33.500930,CNDH
8,Comisión Nacional de los Derechos Humanos,2015,Gasto de obra pública,18.243956,CNDH
9,Comisión Nacional de los Derechos Humanos,2016,Gasto corriente,1688.660270,CNDH


## Presupuesto por concepto. Top 10

In [150]:
dh_concepto = dh.groupby(['DESC_RAMO', 'CICLO', 'DESC_CONCEPTO']).apply(lambda x: x[['aprobado']].sum())\
                                                          .groupby(['DESC_RAMO', 'CICLO'])\
                                                          .apply(lambda y: y.sort_values( 'aprobado',ascending=False).iloc[1:10])
dh_concepto.index = dh_concepto.index.droplevel([2,3])

In [153]:
dh_concepto.to_csv('presup_top10_concepto_ramo_year.csv', index=False)

In [112]:
df.query('CICLO==2014')['MONTO_APROBADO'].sum()

df.DESC_PP.isnull().sum()

0